In [1]:
import os
import json
import numpy
import datetime
import pandas as pd
import pymysql
import requests
import requests.exceptions

import pymongo
from sqlalchemy import create_engine

### Data for CSV file obtained from https://www.kaggle.com/datasets/yasserh/wine-quality-dataset ###

In [47]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "poggers"
db_name = 'wineqt'

In [77]:
def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, data, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df = pd.read_csv(data_file)
    df.to_sql(table_name, con=connection, if_exists='replace')
    connection.close()
    
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    return dframe

def save_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace')
    connection.close()

In [88]:
#Convert file data types
directory = 'Data' #insert file directory name here
dat = 'WineQT.csv' #insert datafile you want to process here
#Create Database Table
insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, dat, table_name)
desired_type = '.json' # insert the data type you want to convert to here
table_name = 'wine_qt' #type desired table name here
data_dir = os.path.join(os.getcwd(), directory)
data_file = os.path.join(data_dir, dat)
file_name, file_extension = os.path.splitext(data_file)
str(file_name)
str(file_extension)
if file_extension == '.csv':
    if desired_type == '.json':
        try:
            df = pd.read_csv(data_file)
            df.head()
            df.to_json(orient="table")
        except:
            print("Unable to find data file.")
    elif desired_type == '.sql':
        try:
            df = insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, data_file, table_name)
        except:
            print("Unable to convert to SQL database table.")
if file_extension == '.json':
    if desired_type == '.csv':
        try:
            with open('jsonfile.json') as input_file:
                df = pd.read_json(input_file)
            df.to_csv()
        except:
            print("Unable to find data file.")
    elif desired_type == '.sql':
        try:
            df = insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, data_file, table_name)
        except:
            print("Unable to convert to SQL database table.")

In [89]:
# Select SQL Query for the Dataset
sql_query = """
    SELECT * from wineqt.wine_qt;
"""
df = get_dataframe(user_id, pwd, host_name, db_name, sql_query)
df.drop('index', axis=1, inplace=True)
# Drop index column as it isn't a necessary column
df.rename(columns={"fixed acidity":"fixed_acidity","volatile acidity":"volatile_acidity","citric acid":"citric_acid","residual sugar":"residual_sugar","free sulfur dioxide":"free_sulfur_dioxide","total sulfur dioxide":"total_sulfur_dioxide"},inplace=True)
# Rename columns names with underscores to make calling them in SQL Queries much easier
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [90]:
number_of_records = df.shape[0] 
# Get number of records from df and print the number
number_of_columns = df.shape[1] 
# Get number of columns
print("The number of records is:",number_of_records)
print("The number of columns is:",number_of_columns)

The number of records is: 1143
The number of columns is: 13


In [91]:
if desired_type == '.csv':
    df.to_csv("WineQT_Mod.csv", index=False)
if desired_type == '.json':
    df.to_json("WineQT_Mod.json", orient='table')
if desired_type == '.sql':
    save_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, 'wineqt_mod')

In [92]:
### API ###
def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"
        
    return result

def get_api_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)
        
    return response.json()

In [93]:
response_type = ['json', 'dataframe']
link2 = 'https://api.coinlore.net/api/coin/markets/?id=90' ### Link for API
API = get_api_response(link2, response_type[1])

In [94]:
### Drop Price as it is not in USD and Time because it is the time from when the API started keeping data
API.drop(columns = ['time','price'], inplace=True)
API.rename(columns = {'name':'Market','base':'Cryptocurrency'}, inplace=True)
API

,Market,Cryptocurrency,quote,price_usd,volume,volume_usd
0,Binance,BTC,USDT,45932.800000,6.494519e+04,2.983115e+09
1,Bitcoin.com,BTC,USD,45927.380000,5.285143e+04,2.427327e+09
2,HitBTC,BTC,USDT,45920.260000,5.269170e+04,2.419616e+09
3,Huobi,BTC,USDT,46025.150000,2.512223e+04,1.156254e+09
4,Coinbase Pro,BTC,USD,45985.640000,1.826454e+04,8.399064e+08
5,Lbank,BTC,USDT,45918.630000,1.681701e+04,7.722143e+08
6,FTX,BTC,USD,46013.000000,1.609418e+04,7.405416e+08
7,ZB.com,BTC,USDT,45863.120000,1.607031e+04,7.370348e+08
8,IDCM,BTC,USDT,45898.940000,1.508157e+04,6.922280e+08
9,Indoex,BTC,USD,46054.681634,1.338568e+04,6.164730e+08


In [95]:
number_of_records = API.shape[0] 
# Get number of records from df and print the number
number_of_columns = API.shape[1] 
# Get number of columns
print("The number of records is:",number_of_records)
print("The number of columns is:",number_of_columns)

The number of records is: 50
The number of columns is: 6


In [96]:
### Create 'cryptocurrency database' ###
crydb_name = 'cryptocurrency'
crytable = "crypto_stock"
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
sqlEngine.execute(f"DROP DATABASE IF EXISTS `{crydb_name}`;")
sqlEngine.execute(f"CREATE DATABASE `{crydb_name}`;")
sqlEngine.execute(f"USE {crydb_name};")
insert_sqlalchemy_dataframe(user_id, pwd, host_name, crydb_name, API, crytable)

In [99]:
## Save API Results to Desired Data Type ##
if desired_type == '.csv':
    API.to_csv("Crypto_Mod.csv",index=False)
if desired_type == '.json':
    API.to_json("Crypto_Mod.json",orient='table')
if desired_type == '.sql':
    save_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, API, table_name)